In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
#  Load dataset
df = pd.read_csv('CO2Dn.csv', nrows=10000, encoding='utf-8-sig')

# Extract target variables
y = df[['CO2']].copy()
X = df.drop(['Date', 'SST','RAD','CO2'], axis=1).copy()

# Impute missing values in X and y using the mean
X.fillna(X.mean(), inplace=True)
y.fillna(y.mean(), inplace=True)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

#  Perform RFE
# Initialize the RFE selector with a Linear Regression model and specify the number of features to select
rfe_selector = RFE(estimator=LinearRegression(), n_features_to_select=5)

# Fit the RFE selector to the training data
rfe_selector.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[rfe_selector.support_]

print("Selected features by RFE:")
print(selected_features)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor


# Step 1: Define Models and Parameter Grids

# Linear Regression (no hyperparameters to tune with GridSearchCV)
#lr = MultiOutputRegressor(LinearRegression())
#lr_param_grid = {}

# KNN Regressor
knn = MultiOutputRegressor(KNeighborsRegressor())
knn_param_grid = {'estimator__n_neighbors': [3, 5, 7, 9]}

# Random Forest Regressor
rf = MultiOutputRegressor(RandomForestRegressor(random_state=42))
rf_param_grid = {'estimator__n_estimators': [50, 100, 150],
                 'estimator__max_depth': [None, 10, 20]}

# Gradient Boosting Regressor
gbr = MultiOutputRegressor(GradientBoostingRegressor(random_state=42))
gbr_param_grid = {'estimator__n_estimators': [50, 100, 150],
                  'estimator__learning_rate': [0.01, 0.1,],
                  'estimator__max_depth': [3, 5,]}


# XGBoost Regressor
xgb = MultiOutputRegressor(XGBRegressor(random_state=42))
xgb_param_grid = {'estimator__n_estimators': [50, 100, 150],
                  'estimator__learning_rate': [0.01, 0.1, ],
                  'estimator__max_depth': [3, 5,]}

# Ridge Regressor
ridge = MultiOutputRegressor(Ridge(random_state=42))
ridge_param_grid = {'estimator__alpha': [0.1, 1.0, 10.0]}

# LASSO Regressor
lasso = MultiOutputRegressor(Lasso(random_state=42))
lasso_param_grid = {'estimator__alpha': [0.1, 1.0, 10.0]}

# SVR (Support Vector Regressor) - Note: SVR can be computationally expensive
svr = MultiOutputRegressor(SVR())
svr_param_grid = {'estimator__C': [0.1, 1.0, 10.0],
                  'estimator__epsilon': [0.01, 0.1, 0.2]}

# MLP Regressor (Neural Network)
mlp = MultiOutputRegressor(MLPRegressor(random_state=42))
mlp_param_grid = {'estimator__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
                  'estimator__activation': ['relu', 'tanh',],
                  'estimator__solver': ['adam', 'sgd'],
                  'estimator__learning_rate_init': [0.001, 0.01, 0.1],
                  'estimator__max_iter': [1000, 2000, 3000]}


models_grid = [
   # (lr, lr_param_grid, 'Linear Regression'),
    (knn, knn_param_grid, 'KNN'),
    (rf, rf_param_grid, 'Random Forest'),
    (gbr, gbr_param_grid, 'Gradient Boosting'),
    (xgb, xgb_param_grid, 'XGBoost'),
    (ridge, ridge_param_grid, 'Ridge'),
    (lasso, lasso_param_grid, 'LASSO'),
    (svr, svr_param_grid, 'SVR'),
    (mlp, mlp_param_grid, 'MLP'),
]

In [ ]:
# Step 2: Implement Grid Search with Cross-Validation and Capture All Metrics

optimized_models = {}
results_list = []

for model, param_grid, name in models_grid:
    print(f"Running GridSearchCV for {name}...")
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    optimized_models[name] = best_model

    print(f"Best parameters for {name}: {grid_search.best_params_}")

    # Evaluate the best model on the train, validation, and test sets
    y_train_pred = best_model.predict(X_train)
    y_val_pred = best_model.predict(X_val)
    y_test_pred = best_model.predict(X_test)


    # Calculate metrics for each output on train, validation, and test sets
    model_results = {'Model': name}
    for i, col in enumerate(y_test.columns): # Assuming the same columns for train, val, test
        model_results[f'{col}_Train_RMSE'] = np.sqrt(mean_squared_error(y_train[col], y_train_pred[:, i]))
        model_results[f'{col}_Train_R2'] = r2_score(y_train[col], y_train_pred[:, i])
        model_results[f'{col}_Train_MAE'] = mean_absolute_error(y_train[col], y_train_pred[:, i])
        model_results[f'{col}_Train_MAPE'] = mean_absolute_percentage_error(y_train[col], y_train_pred[:, i])

        model_results[f'{col}_Val_RMSE'] = np.sqrt(mean_squared_error(y_val[col], y_val_pred[:, i]))
        model_results[f'{col}_Val_R2'] = r2_score(y_val[col], y_val_pred[:, i])
        model_results[f'{col}_Val_MAE'] = mean_absolute_error(y_val[col], y_val_pred[:, i])
        model_results[f'{col}_Val_MAPE'] = mean_absolute_percentage_error(y_val[col], y_val_pred[:, i])


        model_results[f'{col}_Test_RMSE'] = np.sqrt(mean_squared_error(y_test[col], y_test_pred[:, i]))
        model_results[f'{col}_Test_R2'] = r2_score(y_test[col], y_test_pred[:, i])
        model_results[f'{col}_Test_MAE'] = mean_absolute_error(y_test[col], y_test_pred[:, i])
        model_results[f'{col}_Test_MAPE'] = mean_absolute_percentage_error(y_test[col], y_test_pred[:, i])


    results_list.append(model_results)

print("Grid search optimization and evaluation completed.")

In [ ]:
# Step 3: Compare Model Performance and Export All Metrics

results_df_optimized_all_metrics = pd.DataFrame(results_list)
print("Optimized Model Performance (Train, Validation, and Test Sets):")
print(results_df_optimized_all_metrics)

# Export all evaluation metrics to CSV in the CO2 folder
results_df_optimized_all_metrics.to_csv('CO2D/optimized_model_performance_all_metrics.csv', index=False)
print("Optimized model performance (all metrics) exported to CO2D/optimized_model_performance_all_metrics.csv")

In [ ]:
# Step 4: Generate Predictions for All Models

all_predictions = {}

for name, model in optimized_models.items():
    print(f"Generating predictions for {name}...")
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)

    all_predictions[name] = {
        'train': y_train_pred,
        'val': y_val_pred,
        'test': y_test_pred
    }

print("Prediction generation completed.")

In [ ]:
# Step 5: Retrieve Dates for Each Set

train_dates = df.loc[X_train.index, 'Date']
val_dates = df.loc[X_val.index, 'Date']
test_dates = df.loc[X_test.index, 'Date']

print("Dates retrieved for train, validation, and test sets.")

In [ ]:
# Step 6: Prepare and Export DataFrames

for model_name, predictions in all_predictions.items():
    print(f"Exporting predictions for {model_name}...")

    # Train set
    train_df_export = pd.DataFrame({
        'Date': train_dates,
        'Actual_CO2': y_train['CO2'],
        'Predicted_CO2': predictions['train'].flatten() # Flatten in case of multi-output but single target
    })
    train_df_export.to_csv(f'CO2D/{model_name.replace(" ", "_")}_train_predictions.csv', index=False)

    # Validation set
    val_df_export = pd.DataFrame({
        'Date': val_dates,
        'Actual_CO2': y_val['CO2'],
        'Predicted_CO2': predictions['val'].flatten()
    })
    val_df_export.to_csv(f'CO2D/{model_name.replace(" ", "_")}_val_predictions.csv', index=False)

    # Test set
    test_df_export = pd.DataFrame({
        'Date': test_dates,
        'Actual_CO2': y_test['CO2'],
        'Predicted_CO2': predictions['test'].flatten()
    })
    test_df_export.to_csv(f'CO2D/{model_name.replace(" ", "_")}_test_predictions.csv', index=False)

print("Prediction export completed.")

# **Part2**

In [ ]:
# Step 1: Select Base Models

# Sort the results by Test R2 in descending order
sorted_models = results_df_optimized_all_metrics.sort_values(by='CO2_Test_R2', ascending=False)

# Select the top 3 models
selected_base_model_names = sorted_models['Model'].head(3).tolist()


print("Top 3 base models selected for stacking:")
print(selected_base_model_names)

# Retrieve the actual model objects from the optimized_models dictionary
selected_base_models = [(name, optimized_models[name]) for name in selected_base_model_names]

In [ ]:

# Step 2: Prepare Data for Stacking

from sklearn.model_selection import KFold

# Define the number of folds
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

print(f"Training data split into {n_splits} folds for stacking.")

In [ ]:
# Step 3: Train Base Models and Generate Out-of-Fold Predictions

oof_train_preds = np.zeros((X_train.shape[0], len(selected_base_models)))

for i, (name, model) in enumerate(selected_base_models):
    print(f"Generating out-of-fold predictions for {name}...")
    oof_preds = np.zeros((X_train.shape[0],))
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_val_fold, y_val_fold = X_train.iloc[val_idx], y_train.iloc[val_idx]

        model.fit(X_train_fold, y_train_fold)
        oof_preds[val_idx] = model.predict(X_val_fold).flatten() # Flatten in case of multi-output but single target

    oof_train_preds[:, i] = oof_preds

print("Out-of-fold prediction generation completed.")

In [ ]:
# Step 4: Generate Test Predictions from Base Models

test_preds_base_models = np.zeros((X_test.shape[0], len(selected_base_models)))

for i, (name, model) in enumerate(selected_base_models):
    print(f"Generating test predictions for {name}...")
    # Train the base model on the full training data
    model.fit(X_train, y_train)
    # Generate predictions on the test set
    test_preds_base_models[:, i] = model.predict(X_test).flatten() # Flatten in case of multi-output but single target

print("Test prediction generation from base models completed.")

In [ ]:
# Step 5: Train Ridge as Meta-Learner

from sklearn.linear_model import Ridge

# Define Ridge Regressor as the meta-learner
meta_learner_ridge = Ridge(random_state=42)

# Train the meta-learner on the out-of-fold predictions
meta_learner_ridge.fit(oof_train_preds, y_train)


print("Ridge meta-learner training completed.")

In [ ]:
# Step 6: Generate Final Predictions

stacked_test_preds = meta_learner_ridge.predict(test_preds_base_models)

print("Final stacked model predictions generated.")

In [ ]:
# Step 7: Evaluate Stacked Model

stacked_test_rmse = np.sqrt(mean_squared_error(y_test, stacked_test_preds))
stacked_test_r2 = r2_score(y_test, stacked_test_preds)
stacked_test_mae = mean_absolute_error(y_test, stacked_test_preds)
stacked_test_mape = mean_absolute_percentage_error(y_test, stacked_test_preds)

print("Stacked Model Test RMSE:", stacked_test_rmse)
print("Stacked Model Test R2:", stacked_test_r2)
print("Stacked Model Test MAE:", stacked_test_mae)
print("Stacked Model Test MAPE:", stacked_test_mape)

# Step 8: Compare Stacked Model Performance

#stacked_results = {
 #   'Model': 'Stacked Model (Ridge Meta-Learner)',
  #  'CO2_Train_RMSE': np.nan, # Not directly calculated in this stacking approach
   # 'CO2_Train_R2': np.nan,   # Not directly calculated
  #  'CO2_Train_MAE': np.nan,  # Not directly calculated
   # 'CO2_Train_MAPE': np.nan, # Not directly calculated
    #'CO2_Val_RMSE': np.nan,   # Not directly calculated
    #'#CO2_Val_R2': np.nan,     # Not directly calculated
    #'CO2_Val_MAE': np.nan,    # Not directly calculated
    #'CO2_Val_MAPE': np.nan,   # Not directly calculated
    #'CO2_Test_RMSE': stacked_test_rmse,
    #'CO2_Test_R2': stacked_test_r2,
    #'CO2_Test_MAE': stacked_test_mae,
    #'CO2_Test_MAPE': stacked_test_mape
#}

#results_df_optimized_all_metrics = pd.concat([results_df_optimized_all_metrics, pd.DataFrame([stacked_results])], ignore_index=True)

#print("\nOptimized Model Performance (Including Stacked Model):")
#print(results_df_optimized_all_metrics)

In [ ]:
# Step 9: Generate Stacked Predictions for Train and Validation

# Generate base model predictions on the train set (using base models trained on the full train set)
train_preds_base_models = np.zeros((X_train.shape[0], len(selected_base_models)))
for i, (name, model) in enumerate(selected_base_models):
    train_preds_base_models[:, i] = model.predict(X_train).flatten()

# Generate stacked predictions on the train set using the Ridge meta-learner
stacked_train_preds = meta_learner_ridge.predict(train_preds_base_models)


# Generate base model predictions on the validation set (using base models trained on the full train set)
val_preds_base_models = np.zeros((X_val.shape[0], len(selected_base_models)))
for i, (name, model) in enumerate(selected_base_models):
    val_preds_base_models[:, i] = model.predict(X_val).flatten()

# Generate stacked predictions on the validation set using the Ridge meta-learner
stacked_val_preds = meta_learner_ridge.predict(val_preds_base_models)

print("Stacked predictions generated for train and validation sets.")

In [ ]:
# Step 10: Prepare DataFrames for Stacked Predictions

# Train set DataFrame
stacked_train_df_export = pd.DataFrame({
    'Date': train_dates,
    'Actual_CO2': y_train['CO2'],
    'Predicted_CO2': stacked_train_preds.flatten() # Flatten in case of multi-output but single target
})

# Validation set DataFrame
stacked_val_df_export = pd.DataFrame({
    'Date': val_dates,
    'Actual_CO2': y_val['CO2'],
    'Predicted_CO2': stacked_val_preds.flatten()
})

# Test set DataFrame
stacked_test_df_export = pd.DataFrame({
    'Date': test_dates,
    'Actual_CO2': y_test['CO2'],
    'Predicted_CO2': stacked_test_preds.flatten()
})

print("DataFrames for stacked predictions created.")

In [ ]:
# Step 11: Export Stacked Prediction DataFrames to CSV

stacked_train_df_export.to_csv('CO2D/stacked_model_ridge_train_predictions.csv', index=False)
print("Stacked train predictions exported to CO2/stacked_model_ridge_train_predictions.csv")

stacked_val_df_export.to_csv('CO2D/stacked_model_ridge_val_predictions.csv', index=False)
print("Stacked validation predictions exported to CO2/stacked_model_ridge_val_predictions.csv")

stacked_test_df_export.to_csv('CO2D/stacked_model_ridge_test_predictions.csv', index=False)
print("Stacked test predictions exported to CO2/stacked_model_ridge_test_predictions.csv")

In [ ]:
#step 12 Calculate and display evaluation metrics for the stacked model on Train, Validation, and Test sets

# Train set metrics
stacked_train_rmse = np.sqrt(mean_squared_error(y_train, stacked_train_preds))
stacked_train_r2 = r2_score(y_train, stacked_train_preds)
stacked_train_mae = mean_absolute_error(y_train, stacked_train_preds)
stacked_train_mape = mean_absolute_percentage_error(y_train, stacked_train_preds)

print("Stacked Model Train RMSE:", stacked_train_rmse)
print("Stacked Model Train R2:", stacked_train_r2)
print("Stacked Model Train MAE:", stacked_train_mae)
print("Stacked Model Train MAPE:", stacked_train_mape)
print("-" * 30)

# Validation set metrics
stacked_val_rmse = np.sqrt(mean_squared_error(y_val, stacked_val_preds))
stacked_val_r2 = r2_score(y_val, stacked_val_preds)
stacked_val_mae = mean_absolute_error(y_val, stacked_val_preds)
stacked_val_mape = mean_absolute_percentage_error(y_val, stacked_val_preds)

print("Stacked Model Val RMSE:", stacked_val_rmse)
print("Stacked Model Val R2:", stacked_val_r2)
print("Stacked Model Val MAE:", stacked_val_mae)
print("Stacked Model Val MAPE:", stacked_val_mape)
print("-" * 30)

# Test set metrics (already calculated, but displaying again for completeness)
print("Stacked Model Test RMSE:", stacked_test_rmse)
print("Stacked Model Test R2:", stacked_test_r2)
print("Stacked Model Test MAE:", stacked_test_mae)
print("Stacked Model Test MAPE:", stacked_test_mape)

# **SHAP Explanation**

In [ ]:
%pip install shap

In [ ]:
# Step 2: Define Stacked Model Prediction Function

def stacked_model_predict(X):
    """
    Prediction function for the stacked model that takes original features (X)
    and returns the final stacked predictions.
    """
    # Ensure X is a pandas DataFrame to use .iloc for indexing in base models
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=X_train.columns) # Assuming column names are consistent

    # Generate predictions from base models
    base_model_preds = np.zeros((X.shape[0], len(selected_base_models)))
    for i, (name, model) in enumerate(selected_base_models):
        base_model_preds[:, i] = model.predict(X).flatten()

    # Generate final predictions using the meta-learner
    final_predictions = meta_learner_ridge.predict(base_model_preds)

    return final_predictions.flatten() # Return flattened predictions

In [ ]:
# Step 3: Prepare Data for SHAP and Initialize Explainer

import shap

# Select data subsets for explanation
# Using the full datasets as requested. Be aware that Kernel SHAP can be computationally intensive.
X_train_sample = X_train # Use full training data for SHAP explanation
X_val_sample = X_val   # Use full validation data for SHAP explanation
X_test_sample = X_test   # Use full test data for SHAP explanation


# Use a background dataset for the explainer (e.g., a small sample of the training data)
# A smaller background dataset is generally sufficient and helps reduce computation time.
background_data = X_train.sample(min(100, len(X_train)), random_state=42)


# Initialize the Kernel SHAP explainer
# Passing the prediction function and the background dataset
explainer = shap.KernelExplainer(stacked_model_predict, background_data)

print("Data subsets prepared and SHAP Kernel Explainer initialized.")

In [ ]:
# Step 4: Compute SHAP Values

print("Computing SHAP values for train sample...")
shap_values_train = explainer.shap_values(X_train_sample)

print("Computing SHAP values for validation sample...")
shap_values_val = explainer.shap_values(X_val_sample)

print("Computing SHAP values for test sample...")
shap_values_test = explainer.shap_values(X_test_sample)

print("SHAP value computation completed.")

In [ ]:
# Step 5: Generate SHAP Summary Plot (for Test Sample)

import matplotlib.pyplot as plt
import os

# Create the CO2 folder if it doesn't exist
if not os.path.exists('CO2'):
    os.makedirs('CO2')

print("Generating SHAP summary plot (bar) for the test sample and exporting to JPEG...")
# Use plot_type="bar" for overall feature importance
shap.summary_plot(shap_values_test, X_test_sample, plot_type="bar", show=False)
plt.title("SHAP Feature Importance (Test Sample)")
plt.savefig('CO2D/shap_feature_importance_bar.jpeg', dpi=350, bbox_inches='tight')
plt.show() # Display the plot in the notebook
plt.close() # Close the plot to prevent it from displaying twice

print("Generating SHAP summary plot (default) for the test sample and exporting to JPEG...")
# Default summary plot showing impact and direction
shap.summary_plot(shap_values_test, X_test_sample, show=False)
plt.title("SHAP Summary Plot (Test Sample)")
plt.savefig('CO2D/shap_summary_plot.jpeg', dpi=350, bbox_inches='tight')
plt.show() # Display the plot in the notebook
plt.close() # Close the plot

print("SHAP summary plots generated and exported to CO2D folder.")

In [ ]:
# Find the instance with the median predicted value in the test set

# Calculate the median of the stacked test predictions
median_prediction = np.median(stacked_test_preds)

# Find the index of the instance in the test set with the prediction closest to the median
# We find the absolute difference between each prediction and the median prediction,
# and then get the index of the minimum difference.
closest_instance_index_in_test_sample = np.argmin(np.abs(stacked_test_preds - median_prediction))

# Get the actual instance data from the test sample using this index
instance_to_explain_median = X_test_sample.iloc[[closest_instance_index_in_test_sample]]

# Get the SHAP values for this specific instance from the pre-computed SHAP values for the test sample
shap_values_median_instance = shap_values_test[closest_instance_index_in_test_sample]

print(f"Generating SHAP force plot for the instance with a prediction closest to the median predicted value ({median_prediction:.2f})...")

# Visualize the force plot for the median instance
shap.initjs() # Initialize JavaScript visualization in Colab
shap.force_plot(explainer.expected_value, shap_values_median_instance, instance_to_explain_median)

In [ ]:
# Compare Stacked Model Performance with Base Models on Validation and Test Sets

# Extract relevant columns for base models from the results_df_optimized_all_metrics
base_model_comparison = results_df_optimized_all_metrics[['Model', 'CO2_Train_RMSE', 'CO2_Train_R2', 'CO2_Train_MAE', 'CO2_Train_MAPE',
                                                           'CO2_Val_RMSE', 'CO2_Val_R2', 'CO2_Val_MAE', 'CO2_Val_MAPE',
                                                           'CO2_Test_RMSE', 'CO2_Test_R2', 'CO2_Test_MAE', 'CO2_Test_MAPE']].copy()

# Create a DataFrame for the stacked model's performance
stacked_model_comparison = pd.DataFrame([{
    'Model': 'Stacked Model (Ridge Meta-Learner)',
    'CO2_Train_RMSE': stacked_train_rmse,
    'CO2_Train_R2': stacked_train_r2,
    'CO2_Train_MAE': stacked_train_mae,
    'CO2_Train_MAPE': stacked_train_mape,
    'CO2_Val_RMSE': stacked_val_rmse,
    'CO2_Val_R2': stacked_val_r2,
    'CO2_Val_MAE': stacked_val_mae,
    'CO2_Val_MAPE': stacked_val_mape,
    'CO2_Test_RMSE': stacked_test_rmse,
    'CO2_Test_R2': stacked_test_r2,
    'CO2_Test_MAE': stacked_test_mae,
    'CO2_Test_MAPE': stacked_test_mape
}])

# Concatenate the base model and stacked model results
comparison_df = pd.concat([base_model_comparison, stacked_model_comparison], ignore_index=True)

# Sort by Test R2 for easier comparison
comparison_df = comparison_df.sort_values(by='CO2_Test_R2', ascending=False)

print("Performance Comparison: Stacked Model vs. Base Models (Train, Validation, and Test Sets)")
display(comparison_df)

In [ ]:
# Export the final comparison DataFrame to a CSV file
comparison_df.to_csv('CO2D/final_model_comparison.csv', index=False)

print("Final model comparison metrics exported to CO2/final_model_comparison.csv")